In [17]:
%pip install pandas matplotlib altair
import pandas as pd
import altair as alt

Note: you may need to restart the kernel to use updated packages.


In [120]:
import os

# for i in range(0, 1):
import matplotlib.pyplot as plt

def wasm_path_to_benchmark_name(wasm_path: str) -> str:
    splits = wasm_path.split("/")
    if splits[-1] == "benchmark.wasm":
        # E.g. noop/benchmark.wasm -> noop
        return splits[-2]
    else:
        # E.g. libsodium/libsodium-box7.wasm -> libsodium-box7
        return splits[-1].replace(".wasm", "")


def pct_diff(a, b):
    return ((a - b) / (a + b) / 2) * 100


def parse_data(prefix, directory="linux-bench-2"):
    """Yield a chart for each pass for this prefix"""
    df = pd.DataFrame()
    for i in range(1, 100):
        fname = os.path.join(directory, f"{prefix}-{i}.json")
        if not os.path.exists(fname):
            break
        pass_df = pd.read_json(fname)
        pass_df['pass'] = i
        pass_df['prefix'] = prefix
        df = pd.concat([df, pass_df])
    return df


def plot_data(df, pct_quantile=0.25):
    benchmarks = df['wasm'].unique()
    df = df[df['event'] == 'cycles']
    for benchmark in benchmarks:
        sub_df = df[df['wasm'] == benchmark]
        baseline_df = sub_df[sub_df['prefix'].str.contains('baseline')]
        other_df = sub_df[~sub_df['prefix'].str.contains('baseline')]
        prefixes = sub_df['prefix'].unique()
        pct_change_txt = ""
        if len(prefixes) > 1:
            pct_change_text = ", change: "
            for prefix in other_df['prefix'].unique():
                other_df = sub_df[sub_df['prefix'] == prefix]
                baseline_quant = baseline_df['count'].quantile(pct_quantile)
                other_quant = other_df['count'].quantile(pct_quantile)
                diff = pct_diff(other_quant, baseline_quant)
                pct_change_txt += f", {prefix}={diff:+.2f}%"
        iterations_per_pass = len(sub_df['iteration'].unique())
        benchmark_name = wasm_path_to_benchmark_name(benchmark)
        chart = alt.Chart(sub_df).mark_boxplot().encode(
            x=alt.X('pass:N'), 
            y=alt.Y('count:Q', title='Count (Cycles)'),
            column="prefix:N",
            color=alt.Color("prefix:N").legend(None),
        ).properties(
            title=f"{benchmark_name}, iterations/pass={iterations_per_pass}{pct_change_txt}",
        )
        yield chart


In [ ]:
import functools

# df_baseline = parse_data("baseline")
# df_epoch_interrupts = parse_data("epoch-interrution-y")
# df_fuel = parse_data("fuel-enabled")
# df_stack_guard = parse_data("stack-probe")
heapsort_base = parse_data("baseline-heapsort")
heapsort_stack_probe = parse_data("stack-probe-heapsort")
heasport_ee275a899a47adb14031aebc660580378cc2dc06 = parse_data("ee275a899a47adb14031aebc660580378cc2dc06")
df_combined = pd.concat([heapsort_base, heapsort_stack_probe, heasport_ee275a899a47adb14031aebc660580378cc2dc06])
charts = plot_data(df_combined)
single_chart = functools.reduce(lambda c1, c2: c1 & c2, charts)
single_chart.save('sightglass-benchmark-timings.html', embed_options={'renderer': 'svg'})
